<a href="https://colab.research.google.com/github/Deep-Learning-Challenge/challenge-notebooks/blob/master/4.Advanced%20Topics/3.Better%20Predictions/6.%20Learn%20to%20Combine%20Predictions%20with%20Stacked%20Generalization%20Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Learn to Combine Predictions With Stacked Generalization Ensemble

Model averaging is an ensemble technique where multiple submodels contribute equally to a combined prediction. Model averaging can be improved by weighting the contributions of each submodel to the combined prediction by the expected performance of the submodel. This can be extended further by training an entirely new model to learn how to best combine the contributions from each submodel. This approach is called stacked generalization, or stacking for short, and can result in better predictive performance than any single contributing model. In this tutorial, you will discover how to develop a stacked generalization ensemble for deep learning neural networks. After completing this tutorial, you will know:

* Stacked generalization is an ensemble method where a new model learns how to best combine the predictions from multiple existing models.
* Developing a stacking model using neural networks as a submodel and a scikit-learn classifier as the meta-learner.
* How to develop a stacking model where neural network submodels are embedded in a larger stacking ensemble model for training and prediction.

## Stacked Generalization Ensemble

A model averaging ensemble combines the predictions from multiple trained models. A limitation of this approach is that each model contributes the same amount to the ensemble prediction, regardless of how well the model performed. A variation of this approach, called a weighted average ensemble, weighs the contribution of each ensemble member by the trust or expected performance of the model on a hold-out dataset. This allows well-performing models to contribute more and less-well-performing models to contribute less. The weighted average ensemble provides an improvement over the model average ensemble.

A further generalization of this approach is replacing the linear weighted sum (e.g., linear regression) model used to combine the predictions of the submodels with any learning algorithm. This approach is called stacked generalization, or stacking for short. In stacking, an algorithm takes the outputs of submodels as input and attempts to learn how to combine the input predictions best to make a better output prediction. It may be helpful to think of the stacking procedure as having two levels: level 0 and level 1.

* Level 0: The level 0 data is the training dataset inputs, and level 0 models learn to make predictions from this data.
* Level 1: The level 1 data takes the output of the level 0 models as input, and the single level 1 model, or meta-learner, learns to make predictions from this data. 

Unlike a weighted average ensemble, a stacked generalization ensemble can use the set of predictions as a context and conditionally decide to weigh the input predictions differently, potentially resulting in better performance. Interestingly, although stacking is described as an ensemble learning method with two or more level 0 models, it can be used in the case of only a single level, 0 model. In this case, the level 1, or meta-learner, model corrects predictions from the level 0 model.

The meta-learner must be trained on a separate dataset to the examples used to train the level 0 models to avoid overfitting. A simple way that this can be achieved is by splitting the training dataset into a train and validation set. The level 0 models are then trained on the train set. The level 1 model is then trained using the validation set, where the raw inputs are first fed through the level 0 models to get predictions that are used as inputs to the level 1 model. A limitation of the hold-out validation set approach to training a stacking model is that level 0, and level 1 models are not trained on the full dataset.

A more sophisticated approach to training a stacked model involves using k-fold cross-validation to develop the training dataset for the meta-learner model. Each level 0 model is trained using k-fold cross-validation (or even leave-one-out cross-validation for maximum effect); the models are then discarded, but the predictions are retained. This means for each model, there are predictions made by a version of the model that was not trained on those examples, e.g., having hold-out examples, but in this case for the entire training dataset. The predictions are then used as inputs to train the meta-learner. Level 0 models are then trained on the entire training dataset, and together with the meta-learner, the stacked model can be used to make predictions on new data. In practice, it is common to use different algorithms to prepare each of the level 0 models to provide a diverse set of predictions.

It is also common to use a simple linear model to combine the predictions. Because the use of a linear model is common, stacking is more recently referred to as model blending or simply blending, especially in machine learning competitions.

A stacked generalization ensemble can be developed for regression and classification problems. In the case of classification problems, better results have been seen when using the prediction of class probabilities as input to the meta-learner instead of class labels.

Now that we are familiar with stacked generalization, we can develop a stacked deep learning model through a case study.

## Stacked Generalization Ensemble Case Study

In this section, we will demonstrate how to use the stacking ensemble to reduce the variance of an MLP on a simple multiclass classification problem. This example provides a template for applying the stacking ensemble to your neural network for classification and regression problems.

### Multiclass Classification Problem

We will use a small multiclass classification problem as the basis to demonstrate the stacking ensemble. The scikit-learn class provides the `make_blobs()` function that can be used to create a multiclass classification problem with the prescribed number of samples, input variables, classes, and variance of samples within a class. We use this problem with 1,000 examples, with input variables (to represent the x and y coordinates of the points) and a standard deviation of 2.0 for points within each group. We will use the same random state (seed for the pseudorandom number generator) to ensure that we always get the same data points.

In [ ]:
from sklearn.datasets import make_blobs

# generate 2d classification dataset
X, y = make_blobs(n_samples=1000, centers=3, n_features=2, cluster_std=2, random_state=2)

The results are the input and output elements of a dataset that we can model. In order to get a feeling for the complexity of the problem, we can graph each point on a two-dimensional scatter plot and color each point by class value. The complete example is listed below.

In [ ]:
# scatter plot of blobs dataset
from sklearn.datasets import make_blobs
from matplotlib import pyplot
from numpy import where

# generate 2d classification dataset
X, y = make_blobs(n_samples=1000, centers=3, n_features=2, cluster_std=2, random_state=2)

# scatter plot for each class value
for class_value in range(3):
    # select indices of points with the class label
    row_ix = where(y == class_value)

    # scatter plot for points with a different color
    pyplot.scatter(X[row_ix, 0], X[row_ix, 1])

# show plot
pyplot.show()

Running the example creates a scatter plot of the entire dataset. We can see that the standard deviation of 2.0 means that the classes are not linearly separable (separable by a line), causing many ambiguous points. This is desirable because the problem is non-trivial and will allow a neural network model to find many different *good enough* candidate solutions resulting in a high variance.

### Multilayer Perceptron Model

Before we define a model, we need to contrive an appropriate problem for the stacking ensemble. In our problem, the training dataset is relatively small. Specifically, there is a 10:1 ratio of examples in the training dataset to the holdout dataset. This mimics a situation where we may have a vast number of unlabeled examples and a small number of labeled examples with which to train a model. We will create 1,100 data points from the blobs problem. The model will be trained on the first 100 points, and the remaining 1,000 will be held back in a test dataset, unavailable to the model.

The problem is a multiclass classification problem, and we will model it using a softmax activation function on the output layer. This means that the model will predict a vector with three elements with the probability that the sample belongs to each of the three classes. Therefore, the first step is to one-hot encode the class values.

In [ ]:
# scatter plot of blobs dataset
from sklearn.datasets import make_blobs
from tensorflow.keras.utils import to_categorical

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)

# one hot encode output variable
y = to_categorical(y)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
print(trainX.shape, testX.shape)

Next, we can define and compile the model. The model will expect samples with two input variables. The model then has a single hidden layer with 25 nodes and a rectified linear activation function, an output layer with three nodes to predict the probability of each of the three classes, and a softmax activation function. Because the problem is multiclass, we will use the categorical cross-entropy loss function to optimize the model and the efficient Adam flavor of stochastic gradient descent.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# define model
model = Sequential()
model.add(Dense(25, input_dim=2, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

The model is fit for 500 training epochs, and we will evaluate each epoch on the test set, using the test set as a validation set.

In [ ]:
# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=500, verbose=1)

At the end of the run, we will evaluate the model's performance on both the train and the test sets.

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Then finally, we will plot model loss and accuracy learning curves over each training epoch on both the training and test dataset.

In [ ]:
# plot loss learning curves
pyplot.subplot(211)
pyplot.title('Cross-Entropy Loss', pad=-40)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()

# plot accuracy learning curves
pyplot.subplot(212)
pyplot.title('Accuracy', pad=-40)
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()

pyplot.show()

The complete example is listed below.

In [ ]:
# develop an mlp for blobs dataset
from sklearn.datasets import make_blobs
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from matplotlib import pyplot

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)

# one hot encode output variable
y = to_categorical(y)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# define model
model = Sequential()
model.add(Dense(25, input_dim=2, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=500, verbose=0)

# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

Running the example, it first prints the performance of the final model on the train and test datasets.

**Note:** Your specific results may vary, given the stochastic nature of the learning algorithm. Consider running the example a few times and compare the average performance.

In this case, we can see that the model achieved about 83% accuracy on the training dataset, which we know is optimistic, and about 81% on the test dataset, which we would expect to be more realistic.

A line plot also shows the learning curves for the model accuracy on the train and test sets over each training epoch. We can see that training accuracy is more optimistic over the whole run, as we noted with the final scores.

In [ ]:
# plot loss learning curves
pyplot.subplot(211)
pyplot.title('Cross-Entropy Loss', pad=-40)
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()

# plot accuracy learning curves
pyplot.subplot(212)
pyplot.title('Accuracy', pad=-40)
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()

pyplot.show()

We can now look at using instances of this model as part of a stacking ensemble.

### Train and Save Sub-Models

We will use multiple instances of the same model as level-0 or submodels in the stacking ensemble to keep this example simple. We will also use a holdout validation dataset to train the level-1 or meta-learner in the ensemble. A more advanced example may use different types of MLP models (deeper, wider, etc.) as submodels and train the meta-learner using k-fold cross-validation. In this section, we will train multiple submodels and save them to file for later use in our stacking ensembles. The first step is to create a function that defines and fits an MLP model on the training dataset.

In [ ]:
# fit model on dataset
def fit_model(trainX, trainy):
    # define model
    model = Sequential()
    model.add(Dense(25, input_dim=2, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit model
    model.fit(trainX, trainy, epochs=500, verbose=0)

    return model

Next, we can create a subdirectory to store the models.

In [ ]:
from os import makedirs

# create directory for models
makedirs('models', exist_ok =True)

Finally, we can create multiple instances of the MLP and save each to the models/ subdirectory with a unique filename. In this case, we will create five submodels, but you can experiment with a different number of models and see how it impacts model performance.

```
# fit and save models
n_members = 5
for i in range(n_members):
    # fit model
    model = fit_model(trainX, trainy)

    # save model
    filename = 'models/model_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)
```

We can tie all of these elements together; the complete example of training the submodels and saving them to file is listed below.

In [ ]:
# example of saving sub-models for later use in a stacking ensemble
from sklearn.datasets import make_blobs
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from os import makedirs

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)

# one hot encode output variable
y = to_categorical(y)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# create directory for models
makedirs('models', exist_ok =True)

Running the example creates the models/ subfolder and saves five trained models with unique filenames.

In [ ]:
# fit and save models
n_members = 5
for i in range(n_members):
    # fit model
    model = fit_model(trainX, trainy)

    # save model
    filename = 'models/model_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)

Next, we can look at training a meta-learner to make the best use of the predictions from these submodels.

### Separate Stacking Model

We can now train a meta-learner that will best combine the predictions from the submodels and ideally perform better than any single submodel. The first step is to load the saved models, and we can use the `load_model()` Keras function and create a Python list of loaded models.

In [ ]:
# load models from file
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'

        # load model from file
        model = load_model(filename)

        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)

    return all_models

We can call this function to load our five saved models from the models/ sub-directory.

```
# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
```

It would be useful to know how well the single models perform on the test dataset as we would expect a stacking model to perform better. We can easily evaluate every single model on the training dataset and establish a baseline of performance.

```
# evaluate standalone models on test dataset
for model in members:
    testy_enc = to_categorical(testy)
    _, acc = model.evaluate(testX, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)
```

Next, we can train our meta-learner. This requires two steps:

* Prepare a training dataset for the meta-learner.
* Use the prepared training dataset to fit a meta-learner model.

We will prepare a training dataset for the meta-learner by providing examples from the test set to each submodel and collecting the predictions. In this case, each model will output three predictions for each example for the probabilities that a given example belongs to the three classes. Therefore, the 1,000 examples in the test set will result in five arrays with the shape [1000, 3]. We can combine these arrays into a three-dimensional array with the shape [1000, 5, 3] by using the `dstack()` NumPy function to stack each new set of predictions.

As input for a new model, we will require 1,000 examples with some number of features. Given that we have five models and each model makes three predictions per example, we would have 15 (3 X 5) features for each example provided to the submodels. We can transform the [1000, 5, 3] shaped predictions from the submodels into a [1000, 15] shaped array to be used to train a meta-learner using the `reshape()` NumPy function and flattening the final two dimensions. The `stacked_dataset()` function implements this step.

In [ ]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)

        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))

    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    
    return stackX

Once prepared, we can use this input dataset and the output, or y part, of the test set to train a new meta-learner. In this case, we will train a simple logistic regression algorithm from the scikit-learn library. Logistic regression only supports binary classification, although implementing logistic regression in scikit-learn in the `LogisticRegression` class supports multiclass classification (more than two classes) using a multinomial scheme. The function `fit_stacked_model()` below will prepare the training dataset for the meta-learner by calling the `stacked_dataset()` function, then fit a logistic regression model that is then returned.

In [ ]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)

    # fit standalone model
    model = LogisticRegression(solver='lbfgs', multi_class='multinomial')
    model.fit(stackedX, inputy)

    return model

We can call this function and pass in the list of loaded models and the training dataset.

```
# fit stacked model using the ensemble
model = fit_stacked_model(members, testX, testy)
```

Once fit, we can use the stacked model, including the members and the meta-learner, to make predictions on new data. This can be achieved by first using the submodels to make an input dataset for the meta-learner, e.g., by calling the `stacked_dataset()` function, then predicting with the meta-learner. The `stacked_prediction()` function below implements this.

In [ ]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)

    # make a prediction
    yhat = model.predict(stackedX)

    return yhat

We can use this function to predict new data; in this case, we can demonstrate it by making predictions on the test set.

```
# evaluate model on test set
yhat = stacked_prediction(members, model, testX)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)
```

Tying all of these elements together, the complete example of fitting a linear meta-learner for the stacking ensemble of MLP submodels is listed below.

In [ ]:
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from numpy import dstack

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

# evaluate standalone models on test dataset
for model in members:
    testy_enc = to_categorical(testy)
    _, acc = model.evaluate(testX, testy_enc, verbose=0)
    print('Model Accuracy: %.3f' % acc)

Running the example first loads the submodels into a list and evaluates the performance of each. We can see that the best-performing model is the final model with an accuracy of about 81.8%.

**Note:** Your specific results may vary, given the stochastic nature of the learning algorithm. Consider running the example a few times and compare the average performance.

Next, a logistic regression meta-learner is trained on the predicted probabilities from each submodel on the test set, then the entire stacking model is evaluated on the test set. In this case, we can see that the meta-learner outperformed each submodel on the test set, achieving an accuracy of about 83.1%.

In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, testX, testy)

# evaluate model on test set
yhat = stacked_prediction(members, model, testX)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

### Integrated Stacking Model

When using neural networks as submodels, it may be desirable to use a neural network as a meta-learner. Specifically, the sub-networks can be embedded in a larger multi-headed neural network that then learns how to best combine the predictions from each input submodel. It allows the stacking ensemble to be treated as a single large model. The benefit of this approach is that the outputs of the submodels are provided directly to the meta-learner. Further, it is also possible to update the weights of the submodels in conjunction with the meta-learner model if this is desirable. This can be achieved using the Keras functional interface for developing models.

After the models are loaded as a list, a larger stacking ensemble model can be defined where each of the loaded models is used as a separate input-head to the model. This requires that all layers in each loaded model be marked as not trainable, so the weights cannot be updated when the new larger model is trained. Keras also requires that each layer has a unique name, therefore the names of each layer in each of the loaded models will have to be updated to indicate to which ensemble member they belong.

```
# update all layers in all models to not be trainable
for i in range(len(members)):
    model = members[i]
    for layer in model.layers:
        # make not trainable
        layer.trainable = False
        
        # rename to avoid 'unique layer name' issue
        layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
```

Once the submodels have been prepared, we can define the stacking ensemble model. The input layer for each submodel will be used as a separate input head to this new model. This means that k copies of any input data will have to be provided to the model, where k is the number of input models, in this case, 5. The outputs of each of the models can then be merged. In this case, we will use a simple concatenation merge, where a single 15-element vector will be created from the three-class probabilities predicted by each of the five models. We will then define a hidden layer to interpret this input to the meta-learner and an output layer to make its probabilistic prediction. The `define_stacked_model()` function below implements this and will return a stacked generalization neural network model given a list of trained submodels.

In [ ]:
# define stacked model from multiple member input models
def define_stacked_model(members):
    # update all layers in all models to not be trainable
    for i in range(len(members)):
        model = members[i]
        for layer in model.layers:
            # make not trainable
            layer.trainable = False

            # rename to avoid 'unique layer name' issue
            layer._name = 'ensemble_' + str(i+1) + '_' + layer.name

    # define multi-headed input
    ensemble_visible = [model.input for model in members]

    # concatenate merge output from each model
    ensemble_outputs = [model.output for model in members]
    merge = concatenate(ensemble_outputs)
    hidden = Dense(10, activation='relu')(merge)
    output = Dense(3, activation='softmax')(hidden)
    model = Model(inputs=ensemble_visible, outputs=output)

    # plot graph of ensemble
    plot_model(model, show_shapes=True, to_file='model_graph.png')

    # compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

A plot of the network graph is created when this function is called to give an idea of how the ensemble model fits together.

In [ ]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate, Dense

# define ensemble model
stacked_model = define_stacked_model(members)

Creating the plot requires that `pygraphviz` is installed. If this is a challenge on your workstation, you can comment out the call to the `plot_model()` function.

Once the model is defined, it can be fit. We can fit it directly on the holdout test dataset. Because the submodels are not trainable, their weights will not be updated during training, and only the weights of the new hidden and output layer will be updated. The `fit_stacked_model()` the function below will fit the stacking neural network model for 300 epochs.

In [ ]:
# fit a stacked model
def fit_stacked_model(model, inputX, inputy):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]

    # encode output data
    inputy_enc = to_categorical(inputy)

    # fit model
    model.fit(X, inputy_enc, epochs=300, verbose=0)

We can call this function providing the defined stacking model and the test dataset.

```
# fit stacked model on test dataset
fit_stacked_model(stacked_model, testX, testy)
```

Once fit, we can use the new stacked model to predict new data. This is as simple as calling the `predict()` function on the model. One minor change is that we require k copies of the input data in a list to be provided to the model for each k submodel. The `predict_stacked_model()` function below simplifies this process of predicting with the stacking model.

In [ ]:
# make a prediction with a stacked model
def predict_stacked_model(model, inputX):
    # prepare input data
    X = [inputX for _ in range(len(model.input))]

    # make prediction
    return model.predict(X, verbose=0)

We can call this function to make a prediction for the test dataset and report the accuracy. We would expect the performance of the neural network learner to be better than any individual submodel and perhaps competitive with the linear meta-learner used in the previous section.

```
# make predictions and evaluate
yhat = predict_stacked_model(stacked_model, testX)
yhat = argmax(yhat, axis=1)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)
```

Tying all of these elements together, the complete example is listed below.

In [ ]:
# stacked generalization with neural net meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import concatenate, Dense
from numpy import argmax

# generate 2d classification dataset
X, y = make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)

# split into train and test
n_train = 100
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

# load all models
n_members = 5
members = load_all_models(n_members)
print('Loaded %d models' % len(members))

# define ensemble model
stacked_model = define_stacked_model(members)

# fit stacked model on test dataset
fit_stacked_model(stacked_model, testX, testy)

# make predictions and evaluate
yhat = predict_stacked_model(stacked_model, testX)
yhat = argmax(yhat, axis=1)
acc = accuracy_score(testy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Running the example first loads the five submodels.

**Note:** Your specific results may vary, given the stochastic nature of the learning algorithm. Consider running the example a few times and compare the average performance.

A larger stacking ensemble neural network is defined and fit on the test dataset, then the new model is used to predict the test dataset. We can see that, in this case, the model achieved an accuracy of about 83.8%, outperforming the linear model from the previous section.

## Extensions

This section lists some ideas for extending the tutorial that you may wish to explore.

* **Alternate Meta-Learner**. Update the example to use an alternate meta-learner classifier model to the logistic regression model.
* **Single Level 0 Model**. Update the example to use a single level-0 model and compare the results.
* **Vary Level 0 Models**. Develop a study demonstrating the relationship between test classification accuracy and the number of submodels used in the stacked ensemble.
* **Cross-Validation Stacking Ensemble**. Update the example to use k-fold cross-validation to prepare the training dataset for the meta-learner model.
* **Use Raw Input in Meta-Learner**. Update the example so that the meta-learner algorithms take the raw input data for the sample and the output from the submodels and compare performance.

## Summary

In this tutorial, you discovered how to develop a stacked generalization ensemble for deep learning neural networks. Specifically, you learned:

* Stacked generalization is an ensemble method where a new model learns how to best combine the predictions from multiple existing models.
* How to develop a stacking model using neural networks as a submodel and a scikit-learn classifier as the meta-learner.
* How to develop a stacking model where neural network submodels are embedded in a larger stacking ensemble model for training and prediction.